# Summary

#### This is demo project illustrating a real life company problem where we have to build an automatic script which will read, process and save the results to an excel (xlsx format) file in an automatic way.
#### Here we are given a toy dataset for a company's customer care centre (call centre/help centre) focusing on the chat times and responses by the agents as well as bot.
#### The details regarding the required metrics to be calculated and the format for the output excel file are given in the readme file. 
#### Note: Here we will be demonstrating 2 different ways for calculate metrics 1 and metrics 2.

In [1]:
# Loading Necessary Libraries

import numpy as np
import pandas as pd
import datetime as dt
from calendar import monthrange
import math

#### We will first import the file and explore the data to understand it well and also perform some sanity checks to confirm data integrity.

In [2]:
# Reading excel file using pandas

df = pd.read_excel('Data Dump - Data Analytics Intern Questions.xlsx')

In [3]:
df.head()

,RoomCode,UserId,ClosedBy,ClosingComment,ChatStartTime,ChatEndTime,AgentAssignmentTimestamp,BotFirstResponseTime,AgentFirstResponseTime,CsatScore
0,8528073,e2ff5f48-5f2b-4bb6-894b-fad28843fe0b,System,Closing inactive room,2021-04-02 10:39:34,2021-04-02 11:40:20,-,00:00:00,-,0
1,8674865,362f984a-1487-4fd8-85b2-1e2a94442f9b,System,Closing now,26-02-2021 12:51:37,26-02-2021 12:55:27,-,00:00:00,-,0
2,8543522,4ae0951a-4ba4-4e2f-863b-eb1fab965933,System,Closing inactive room,2021-06-02 12:17:00,2021-06-02 13:17:07,-,00:00:00,-,0
3,8581336,b675563d-8906-4b9d-9368-fda6f21b16d5,System,Closing now,2021-11-02 19:24:24,2021-11-02 19:25:03,-,00:00:01,-,0
4,8613422,b254a88e-4b76-4667-8352-f1583d71da1a,Nisha,Unresponsive Customer,16-02-2021 14:37:17,16-02-2021 14:59:54,16-02-2021 14:39:56,00:00:00,00:00:04,3


#### We observe time data is in two different formats and we will later change and keep in yy/mm//dd

In [4]:
df.shape

(10000, 10)

In [5]:
df.dtypes.value_counts()

object    8
int64     2
dtype: int64

#### We note all the time data is in object format and needs to changed

In [6]:
df['RoomCode'].nunique()

10000

#### We confirm all values of 'RoomCode' are unique as given to us in the description i.e. no duplicates

In [7]:
df['ClosedBy'].unique()

array(['System', 'Nisha', 'Kabir', 'Priti', 'Mohsin', 'Samy', 'Adaa',
       'Vikram', 'Deepa', 'Sandy', 'Manpreet', 'Ellie', 'Maroon',
       'Gurpreet', 'Juvi', 'Eva', 'Vedha', 'Ganesh', 'Jinnie', 'Isha',
       'Simran', 'Rozeta', 'Bot', 'Rohan', 'Norah', 'Bot\n', 'Salman',
       'Rose', 'Jade', 'Megha', 'Pritam', 'Azaan', 'Nayab', 'Geetika',
       'Meher', 'Abbas', 'Anny', 'Elina', 'Mira', 'Zia', 'Avani',
       'Liliana', 'Nadia', 'Lara', 'Quinn', 'Alia', 'Anisha', 'Roop',
       'Kunal', 'Simmi', 'carrie', 'Ayush', 'Yash', 'Keyur', 'Dalphine',
       'Roshni', 'Kesha', 'Priyanka', 'Nikita', 'komi', 'Mary', 'Satish',
       'Serena', 'Rahul', 'Akash', 'Hana', 'Atif', 'Sourav', 'Manisha',
       'Lakshmi', 'Pawanpreet', 'Ajay', 'Aria', 'Murtaza', 'Ruchi',
       'Adil', 'Asif', 'Hayat', 'Gaurav', 'Swaraj', 'Paul', 'Sudha',
       'Diya', 'Rebecca', 'Rusali', 'Sravan', 'Daanish', 'Saif', 'Raman',
       'Krishna', 'Kevin', 'Pratyush', 'Ryla', 'Tia', 'Swati', 'Meeta',
       'Taniy

#### We observe 'Automation System' has three different names ('System', 'Bot', 'Bot\n') and thus needs to be fixed to a single one

In [8]:
df['ClosingComment'].unique()

array(['Closing inactive room', 'Closing now', 'Unresponsive Customer',
       'Completed', 'webhook block failed',
       'Bot closed the conversation. NO transfer'], dtype=object)

## Converting the date time values (object type) with 2 different date-time format to single format of datetime type (yy/mm/dd)

In [9]:
date_col = ['ChatStartTime', 'ChatEndTime', 'AgentAssignmentTimestamp']

In [10]:
for i in date_col:
    date1 = pd.to_datetime(df[i], errors='coerce', format='%Y-%d-%m %H:%M:%S')
    date2 = pd.to_datetime(df[i], errors='coerce', format='%d-%m-%Y %H:%M:%S')
    df['{0}_fix'.format(i)] = date1.fillna(date2)   

#### We added new columnns for our fixed datatime values with column names having a suffix '_fix' to their previous columns names

## Converting Time columns to datetime format

In [11]:
time_cols = ['BotFirstResponseTime','AgentFirstResponseTime']
df[time_cols] = df[time_cols].apply(pd.to_datetime, errors = 'coerce', format='%H:%M:%S')
df[time_cols]

,BotFirstResponseTime,AgentFirstResponseTime
0,1900-01-01 00:00:00,NaT
1,1900-01-01 00:00:00,NaT
2,1900-01-01 00:00:00,NaT
3,1900-01-01 00:00:01,NaT
4,1900-01-01 00:00:00,1900-01-01 00:00:04
...,...,...
9995,1900-01-01 00:00:00,1900-01-01 00:00:13
9996,1900-01-01 00:00:00,NaT
9997,1900-01-01 00:00:00,NaT
9998,1900-01-01 00:00:00,1900-01-01 00:00:07


#### Here we convert and replace the previous dataframe columns having time values (object type) with datetime type. 
#### Note: Although the date here (1900-01-01) is misrepresented, we will leave it be as a dummy value as it will be easier to perform calcualtions on it using datetime library functions. Also, time can be extracted from these values later on as and when required. 

## Changing automation system name to 'System' for all varying names

In [12]:
mask = (df['ClosedBy'] == 'Bot') | (df['ClosedBy'] == 'Bot\n') 
df.loc[mask, 'ClosedBy'] = 'System'
df['ClosedBy'].unique()

array(['System', 'Nisha', 'Kabir', 'Priti', 'Mohsin', 'Samy', 'Adaa',
       'Vikram', 'Deepa', 'Sandy', 'Manpreet', 'Ellie', 'Maroon',
       'Gurpreet', 'Juvi', 'Eva', 'Vedha', 'Ganesh', 'Jinnie', 'Isha',
       'Simran', 'Rozeta', 'Rohan', 'Norah', 'Salman', 'Rose', 'Jade',
       'Megha', 'Pritam', 'Azaan', 'Nayab', 'Geetika', 'Meher', 'Abbas',
       'Anny', 'Elina', 'Mira', 'Zia', 'Avani', 'Liliana', 'Nadia',
       'Lara', 'Quinn', 'Alia', 'Anisha', 'Roop', 'Kunal', 'Simmi',
       'carrie', 'Ayush', 'Yash', 'Keyur', 'Dalphine', 'Roshni', 'Kesha',
       'Priyanka', 'Nikita', 'komi', 'Mary', 'Satish', 'Serena', 'Rahul',
       'Akash', 'Hana', 'Atif', 'Sourav', 'Manisha', 'Lakshmi',
       'Pawanpreet', 'Ajay', 'Aria', 'Murtaza', 'Ruchi', 'Adil', 'Asif',
       'Hayat', 'Gaurav', 'Swaraj', 'Paul', 'Sudha', 'Diya', 'Rebecca',
       'Rusali', 'Sravan', 'Daanish', 'Saif', 'Raman', 'Krishna', 'Kevin',
       'Pratyush', 'Ryla', 'Tia', 'Swati', 'Meeta', 'Taniya'],
      dtype=object

#### Here we have changed the name for all different types/names of automation system in 'ClosedBy' column to 'System'

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   RoomCode                      10000 non-null  int64         
 1   UserId                        10000 non-null  object        
 2   ClosedBy                      10000 non-null  object        
 3   ClosingComment                10000 non-null  object        
 4   ChatStartTime                 10000 non-null  object        
 5   ChatEndTime                   10000 non-null  object        
 6   AgentAssignmentTimestamp      10000 non-null  object        
 7   BotFirstResponseTime          9998 non-null   datetime64[ns]
 8   AgentFirstResponseTime        3722 non-null   datetime64[ns]
 9   CsatScore                     10000 non-null  int64         
 10  ChatStartTime_fix             10000 non-null  datetime64[ns]
 11  ChatEndTime_fix              

#### No null values found for integer types (Datetime columns have some Nat's assigned by us and are ok)

In [14]:
df.describe(datetime_is_numeric = True)

,RoomCode,BotFirstResponseTime,AgentFirstResponseTime,CsatScore,ChatStartTime_fix,ChatEndTime_fix,AgentAssignmentTimestamp_fix
count,1.000000e+04,9998,3722,10000.000000,10000,10000,3722
mean,8.596392e+06,1900-01-01 00:00:00.171709184,1900-01-01 00:00:13.761152768,0.968500,2021-02-14 09:53:29.108499712,2021-02-14 10:21:07.759794688,2021-02-14 10:16:01.131648768
min,7.701262e+06,1900-01-01 00:00:00,1900-01-01 00:00:01,0.000000,2021-01-31 23:05:09,2021-02-01 00:05:47,2021-02-01 09:00:34
25%,8.549444e+06,1900-01-01 00:00:00,1900-01-01 00:00:05,0.000000,2021-02-07 10:35:55,2021-02-07 10:50:30.500000,2021-02-06 18:35:04.749999872
50%,8.595529e+06,1900-01-01 00:00:00,1900-01-01 00:00:07,0.000000,2021-02-13 18:57:41.500000,2021-02-13 19:17:57,2021-02-14 10:08:26.500000
75%,8.644398e+06,1900-01-01 00:00:00,1900-01-01 00:00:11,1.000000,2021-02-21 14:05:54.249999872,2021-02-21 14:37:14,2021-02-20 19:12:07.750000128
max,8.690837e+06,1900-01-01 00:00:37,1900-01-01 00:29:18,5.000000,2021-02-28 23:57:10,2021-03-01 00:39:21,2021-02-28 20:35:48
std,5.485097e+04,NaN,NaN,1.709503,NaN,NaN,NaN


#### Checking min and max values of columns and all seem to be fine and in range

-------

# Chat Metrics (Metric Table 1)

#### Now we will calcualte our first metric table (Chat Metrics) as described in the readme file

#### As most metrics here require a count operation for calculation, we will make use of a general function which can be re-used later for other months also if needed.

In [15]:
# Pre-calculations for metrics 1 before they can be used on the main function

# For calculating Total Incoming Chats
inc_chat = df['RoomCode']

# For calculating Total Unique Users
uni_user = pd.Series(df['UserId'].unique())

# For calculating Total Chats Closed By System/Bot
clsd_bot = df['ClosedBy'].loc[df['ClosedBy'] == 'System']

# Defining a dummy value for calculating Avg Bot Deflection % i.e. count of Closed By / Incoming Chats
bot_def = 0

# For calculating Total Chats Closed By Agents
clsd_user = df['ClosedBy'].loc[df['ClosedBy'] != 'System']

In [16]:
# Function for calculating Table 1 metrics

def metrics1_weekly(name, data, year = 2021, month = 2, day = 1, by = 7):

    weekly_metric = dict()
    num_days = monthrange(year, month)[1]
    by = by    
        
    if type(data) != type(bot_def):
        
        for i in range(math.ceil(num_days/by)):
            weekly_data = df['ChatStartTime_fix'].between(dt.datetime(year, month, day),dt.datetime(year, month, min(day + by, num_days)))
            weekly_metric.update({i : data.loc[weekly_data].count()})
            day = day + by
        result_series[name] = pd.Series(weekly_metric)
    else:
        data = (result_series['Closed By Bot'] / result_series["Incoming Chats"]) * 100
        result_series[name] = data

In [17]:
# Calling the function and saving calculated metrics to a dataframe in desired format

metrics1 = {'Incoming Chats' : inc_chat, 'Unique Users' : uni_user, "Closed By Bot" : clsd_bot, "Bot Deflection %" : bot_def, "Closed By User" : clsd_user}
result_series = dict()

for name, metric in metrics1.items():
    metrics1_weekly(name, metric, 2021,2,1)
    
metrics1_df = pd.DataFrame(result_series)

#Setting Row Names
metrics1_df.index = ["1 Feb - 7 Feb","8 Feb - 14 Feb","15 Feb - 21 Feb","22 Feb - 28 Feb"]
# Setting Multi-Index Column names
metrics1_df.columns = [metrics1_df.columns.values, ["SUM", "SUM", "SUM", "AVERAGE", "SUM"] ]
# Setting Multi-Index name for 'Index' 
metrics1_df.rename_axis(('Week','Overall'), axis = 1, inplace = True)

metrics1_df

Week,Incoming Chats,Unique Users,Closed By Bot,Bot Deflection %,Closed By User
Overall,SUM,SUM,SUM,AVERAGE,SUM
1 Feb - 7 Feb,2763,2603,1789,64.748462,974
8 Feb - 14 Feb,2620,2475,1674,63.893130,946
15 Feb - 21 Feb,2286,2139,1366,59.755031,920
22 Feb - 28 Feb,2036,1909,1185,58.202358,851


---------

# Agent Metrics (Metric Table 2)

#### Now we will calcualte our second metric table (Agent Metrics) as described in the readme file

#### Here as we have to perform different operations for calculating different metrics we will not use function approach (DRY not useful) and a better approach would be to save the segregrated weeks in a dataframe and separately perform the required operations on them.
#### So we will only write a function for segregrating the weeks which can be re-used for different months and different frequencies as desirable
#### Note: Agents here refer to people i.e. not automated system only humans 

In [18]:
# Function for segregrating dataframe 

def segragrate_df(year = 2021, month = 2, day = 1, by = 7):
    
    df['Week_no'] = None
    weekly_metric = dict()
    num_days = monthrange(year, month)[1]
    by = by
    
    for i in range(1, math.ceil(num_days/by)+1):
        weekly_data = df['ChatStartTime_fix'].between(dt.datetime(year, month, day),dt.datetime(year, month, min(day + by, num_days)))
        #df.loc[weekly_data,'Week_no'] = i
        #i = pd.DataFrame(weekly_data)    
        weekly_metric.update({i : df.loc[weekly_data]})

        day = day + by
    
    return weekly_metric 

weekly_dfs = segragrate_df()

#### Saving the segregrated weeks to a dictionary 'weekly_dfs'

In [19]:
# Declaring dictionary variables to store the values weekly and for different metrics

metrics2_ser = dict()
weekly_metric = dict()

In [20]:
# Calculating Total Chats Resolved 

chat_resolved = (df['ClosedBy'] != 'System') & (df['ClosingComment'] == 'Completed')

for i in range(1,5):
    weekly_metric[i] = weekly_dfs[i].loc[chat_resolved,'RoomCode'].count()
metrics2_ser['Chats Resolved'] = pd.Series(weekly_metric)

In [21]:
# Calculating Avg Agent Response Time in seconds

for i in range(1,5):
    weekly_metric[i] = weekly_dfs[i]['AgentFirstResponseTime'].mean().second
metrics2_ser['Avg Agent First Response Time (seconds)'] = pd.Series(weekly_metric)


In [22]:
## Calculating Avg Chat Resolution Time in seconds

for i in range(1,5):
    weekly_metric[i] = (weekly_dfs[i]['ChatEndTime_fix'] - weekly_dfs[i]['ChatStartTime_fix']).mean().total_seconds()
metrics2_ser["Avg Agent Chat Resolution Time (seconds)"] = pd.Series(weekly_metric)


In [23]:
# Calculating Avg Agent CSAT Score

pos_csat = (df['ClosedBy'] != 'System') & ((df['CsatScore'] == 4) | (df['CsatScore'] == 5))
neg_csat = (df['ClosedBy'] != 'System') & ((df['CsatScore'] == 1) | (df['CsatScore'] == 2) | (df['CsatScore'] == 3))

for i in range(1,5):    
    pos_csat_avg = weekly_dfs[i].loc[pos_csat, 'RoomCode'].count()
    neg_csat_avg = weekly_dfs[i].loc[neg_csat, 'RoomCode'].count()
    weekly_metric[i] = pos_csat_avg / (pos_csat_avg + neg_csat_avg)
metrics2_ser["Average Agent CSAT Score"] = pd.Series(weekly_metric)
    

In [24]:
# Calculating Avg Agent CSAT Score during Business Hours [10AM - 5PM]

for i in range(1,5):    
    pos_csat_busn = weekly_dfs[i].set_index("ChatStartTime_fix").between_time('10:00:00','17:00:00').reset_index().loc[pos_csat,"RoomCode"].count()
    neg_csat_busn = weekly_dfs[i].set_index("ChatStartTime_fix").between_time('10:00:00','17:00:00').reset_index().loc[neg_csat,"RoomCode"].count()
    weekly_metric[i] = pos_csat_busn / (pos_csat_busn + neg_csat_busn)
metrics2_ser["Business Hours CSAT [Business Hours: 10AM - 5PM]"] = pd.Series(weekly_metric)

In [25]:
# Saving calculated metrics to a dataframe in a desired format

metrics2_df = pd.DataFrame(metrics2_ser)

#Setting Row Names
metrics2_df.index = ["1 Feb - 7 Feb","8 Feb - 14 Feb","15 Feb - 21 Feb","22 Feb - 28 Feb"]
# Setting Multi-Index Column names
metrics2_df.columns = [metrics2_df.columns.values, ["SUM", "SUM", "AVERAGE", "AVERAGE", "AVERAGE"] ]
# Setting Multi-Index name for 'Index' 
metrics2_df.rename_axis(('Week','Overall'), axis = 1, inplace = True)

metrics2_df

Week,Chats Resolved,Avg Agent First Response Time (seconds),Avg Agent Chat Resolution Time (seconds),Average Agent CSAT Score,Business Hours CSAT [Business Hours: 10AM - 5PM]
Overall,SUM,SUM,AVERAGE,AVERAGE,AVERAGE
1 Feb - 7 Feb,750,13,1669.403184,0.722078,0.796154
8 Feb - 14 Feb,756,14,1684.945801,0.787879,0.799197
15 Feb - 21 Feb,732,13,1590.304899,0.750000,0.816425
22 Feb - 28 Feb,656,13,1734.295677,0.770440,0.815789


# Saving To Excel

#### Now after calculating our metrics, we can finally export them in excel (xlsx) format
#### We will be exporting and also performing some basic to advanced exporting operations (like word-wrap, text centering, adjusting columns widths, merging cells etc) to get the desired output in excel format.

In [26]:
# Save to Excel

writer = pd.ExcelWriter('Chat_Data_Metrics.xlsx', engine = 'xlsxwriter')

metrics1_df.to_excel(writer, 'Chat Metrics', startrow = 4)
metrics2_df.to_excel(writer, 'Agent Metrics', startrow = 4)

workbook  = writer.book

worksheet1 = writer.sheets['Chat Metrics']
worksheet2 = writer.sheets['Agent Metrics']

cell_format = workbook.add_format({'text_wrap' : True})
head_format = workbook.add_format({'text_wrap' : True, 'bold': True, 'border' : 1})

# Function To Adjust Columns Widths across all columns (includes merging of very wide cells widths)
def adj_col_width(metrics,wksheet):

    indx_len = len(metrics.index[0])
    wksheet.set_column(0, 0, (indx_len + 2))

    for idx,column in enumerate(metrics,1):
        column_length = (len(column[0]) + 2)
        col_idx = idx
        if column_length <= 40:
            wksheet.set_column(col_idx, col_idx, column_length, cell_format)
        else:
            num_to_alpha = chr(col_idx + 64 + 1)
            excel_range = num_to_alpha + '4'+ ":" + num_to_alpha + '5'  # Since startrow is 4
            wksheet.merge_range(excel_range, metrics.columns[idx - 1][0], head_format)
            wksheet.set_column(col_idx, col_idx, 35, cell_format)

cell_format.set_align('center')
cell_format.set_align('vcenter')
head_format.set_align('center')
head_format.set_align('vcenter')

adj_col_width(metrics1_df,worksheet1)
adj_col_width(metrics2_df,worksheet2)

writer.save()